In [56]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import warnings
import time
import sys
import datetime
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import BayesianRidge
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import gc
pd.set_option('display.width',None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_info_columns', 500)
np.random.seed(4950)

In [57]:
train = pd.read_csv("./data/pre_train_clip.csv", index_col=0)


test = pd.read_csv("./data/pre_test_clip.csv", index_col=0)

cats = [ 'first_active_month',
        'feature_1', 'feature_2', 'feature_3', 
        'hist_first_year', 
        'hist_first_quarter', 
        'hist_first_month',
        'hist_re_year', 
        'hist_re_quarter', 
        'hist_re_month',
        'hist_now_year', 
        'hist_now_quarter', 
        'hist_now_month',
       ]

In [58]:
data = pd.concat([train, test])    
data.reset_index(drop=True, inplace=True)

for feat in cats:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
#     data[feat][data[feat]>=0] = lbe.fit_transform(data[feat][data[feat]>=0])

In [59]:
train[cats] = data[0:201917][cats]

In [60]:
tmp = data[201917:][cats].reset_index(drop=True)
test[cats] = tmp

In [61]:
for c in cats:
    print(train[c].unique())
    print(test[c].unique())
    print(data[c].unique())

[67 62 57 70 72 58 61 69 59 52 65 63 56 55 71 47 53 68 50 60 48 18 51 54
 27 66 73 64 45 42 38 33 39 31 32 28 43 30 49 44 40 46 26 29 35 36 34 25
 37 19 41 23 21 20  6  9 11  7 10  5 13 24 22 17 12 14  0 15  4  3 16 74
  8 75  1]
[65 62 69 73 49 68 71 63 52 64 53 61 57 72 70 59 67 54 16 37 56 66 58 60
 40 51 55 22 39 43 47 35 34 46 50 38 19 42 45 48 29 28 31 44  8 41 32 27
 36 20 30 33 11 25 74  3 21  7 24 23 26 15 17 12 10  5  6  4 13  9 18 14
  0  1  2]
[67 62 57 70 72 58 61 69 59 52 65 63 56 55 71 47 53 68 50 60 48 18 51 54
 27 66 73 64 45 42 38 33 39 31 32 28 43 30 49 44 40 46 26 29 35 36 34 25
 37 19 41 23 21 20  6  9 11  7 10  5 13 24 22 17 12 14  0 15  4  3 16 74
  8 75  1  2]
[4 3 1 0 2]
[2 1 4 0 3]
[4 3 1 0 2]
[1 0 2]
[2 0 1]
[1 0 2]
[1 0]
[1 0]
[1 0]
[6 5 4 2 3 1 0 7]
[6 4 5 2 3 1 7 0]
[6 5 4 2 3 1 0 7]
[1 0 2 3]
[1 0 2 3]
[1 0 2 3]
[ 5  0  7  8 10 11  9  2  3  1  6  4]
[ 3  0  7 11  6  9  1  2 10  8  5  4]
[ 5  0  7  8 10 11  9  2  3  1  6  4]
[1 0]
[0 1]
[1 0]
[0 2 3 1]
[3 

In [62]:
fold_target = train['target']
fold_target = (fold_target<-20).astype(int)

In [63]:
features = [i for i in train.columns if i not in ['card_id','target','outliers']]
print(len(features),features)

301 ['first_active_month', 'feature_1', 'feature_2', 'feature_3', 'authorized_flag_mean', 'hist_transactions_count', 'hist_is_month_start_mean', 'hist_weekend_mean', 'hist_category_1_mean', 'hist_category_2_nunique', 'hist_category_3_nunique', 'hist_state_id_nunique', 'hist_city_id_nunique', 'hist_subsector_id_nunique', 'hist_merchant_category_id_nunique', 'hist_merchant_id_nunique', 'hist_quarter_nunique', 'hist_month_nunique', 'hist_weekofyear_nunique', 'hist_dayofweek_nunique', 'hist_day_nunique', 'hist_hour_nunique', 'hist_a2p_mean', 'hist_a2p_median', 'hist_a2p_max', 'hist_a2p_min', 'hist_a2p_std', 'hist_p2r_mean', 'hist_p2r_median', 'hist_p2r_max', 'hist_p2r_min', 'hist_p2r_std', 'hist_p2now_mean', 'hist_p2now_median', 'hist_p2now_max', 'hist_p2now_min', 'hist_p2now_std', 'hist_month_lag_mean', 'hist_month_lag_median', 'hist_month_lag_max', 'hist_month_lag_min', 'hist_month_lag_std', 'hist_purchase_amount_sum', 'hist_purchase_amount_mean', 'hist_purchase_amount_median', 'hist_pur

In [64]:
obj_fea = []

for i in features:
    if str(train[i].dtype)=='object':
        print(i)
        obj_fea.append(i)

In [65]:
param = {
         'objective':'regression',
         "metric": 'rmse',
         "boosting": "gbdt",
         'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'max_depth': -1,
         'learning_rate': 0.01,
         "lambda_l1": 0.1,
         "feature_fraction": 0.9,
         "bagging_fraction": 0.9 ,
         "bagging_freq": 1,
   
         "random_state": 1024,
         "verbosity": -1,
}

n_fold = 5

# param ={
#         'task': 'train',
#         'boosting': 'goss',
#         'objective': 'regression',
#         'metric': 'rmse',
#         'learning_rate': 0.01,
#         'subsample': 0.9855232997390695,
#         'max_depth': 7,
#         'top_rate': 0.9064148448434349,
#         'num_leaves': 63,
#         'min_child_weight': 41.9612869171337,
#         'other_rate': 0.0721768246018207,
#         'reg_alpha': 9.677537745007898,
#         'colsample_bytree': 0.5665320670155495,
#         'min_split_gain': 9.820197773625843,
#         'reg_lambda': 8.2532317400459,
#         'min_data_in_leaf': 21,
#         'verbose': -1,
#         'seed':int(2**n_fold),
#         'bagging_seed':int(2**n_fold),
#         'drop_seed':int(2**n_fold)
#         }

In [66]:
# from sklearn.model_selection import RepeatedKFold
# folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2333)
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=4590)
oof_train = np.zeros((len(train),1))
oof_test = np.zeros(len(test))
oof_test_skf = np.zeros((5,len(test),1))
start = time.time()
feature_importance_df = pd.DataFrame()

target = train['target']

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, fold_target)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])
#     trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
#     val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data],
                    verbose_eval=100, early_stopping_rounds = 200)
    oof_train[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration).reshape(-1,1)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    oof_test_skf[fold_,:]= clf.predict(test[features], num_iteration=clf.best_iteration).reshape(-1,1)
#     oof_test += clf.predict(test[features], num_iteration=clf.best_iteration) / 10
    oof_test += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.64415	valid_1's rmse: 3.71792
[200]	training's rmse: 3.55314	valid_1's rmse: 3.67891
[300]	training's rmse: 3.49629	valid_1's rmse: 3.66338
[400]	training's rmse: 3.45431	valid_1's rmse: 3.65498
[500]	training's rmse: 3.4213	valid_1's rmse: 3.64991
[600]	training's rmse: 3.39227	valid_1's rmse: 3.64616
[700]	training's rmse: 3.36714	valid_1's rmse: 3.64377
[800]	training's rmse: 3.34479	valid_1's rmse: 3.64266
[900]	training's rmse: 3.32358	valid_1's rmse: 3.64189
[1000]	training's rmse: 3.30356	valid_1's rmse: 3.64072
[1100]	training's rmse: 3.28473	valid_1's rmse: 3.64052
[1200]	training's rmse: 3.26605	valid_1's rmse: 3.64008
[1300]	training's rmse: 3.24916	valid_1's rmse: 3.64038
Early stopping, best iteration is:
[1183]	training's rmse: 3.26912	valid_1's rmse: 3.63986
fold n°2
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.64704	valid_1's rmse: 

In [67]:
print("CV score: {:<8.5f}".format(mean_squared_error(oof_train, target)**0.5))

CV score: 3.64525 


In [68]:

mean_loss=np.sqrt(mean_squared_error(oof_train.reshape(-1), target))
print(mean_loss)
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = oof_test
sub_df.to_csv("sub/submission_%s.csv"%mean_loss, index=False)

train_prob=pd.DataFrame(oof_train)
train_prob.columns=['class1']
train_prob.to_csv("oof/train_prob_%s.csv"%mean_loss,index=False)

test_prob=pd.DataFrame(oof_test)
test_prob.columns=['class1']
test_prob.to_csv("oof/test_prob_%s.csv"%mean_loss,index=False)



3.6452474956099654


In [69]:
-

Done with   80 of   80 (Spent  25.4 min)

In [70]:
feature_scores = []
for _f in actual_imp_df['feature'].unique():
    f_null_imps_gain = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_gain'].values
    f_act_imps_gain = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_gain'].mean()
    gain_score = np.log(1e-10 + f_act_imps_gain / (1 + np.percentile(f_null_imps_gain, 75)))  # Avoid didvide by zero
    f_null_imps_split = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_split'].values
    f_act_imps_split = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_split'].mean()
    split_score = np.log(1e-10 + f_act_imps_split / (1 + np.percentile(f_null_imps_split, 75)))  # Avoid didvide by zero
    feature_scores.append((_f, split_score, gain_score))

scores_df = pd.DataFrame(feature_scores, columns=['feature', 'split_score', 'gain_score'])



In [71]:
thr_list=scores_df['gain_score'].tolist()
thr_list=[ i for i in set(thr_list)]
thr_list=sorted(thr_list)
thr_list

[-23.025850929940457,
 -2.417094141122722,
 -2.1721172070821346,
 -1.971538422845508,
 -1.9022068000399852,
 -1.5640649367963049,
 -0.7886526792881117,
 -0.5256912688088663,
 -0.49038209815156625,
 -0.45037776427954773,
 -0.1796722117071125,
 -0.051559236558878035,
 -0.03975551469821427,
 7.02673650286135e-05,
 0.11870371605559514,
 0.20464078694392787,
 0.2264183049115753,
 0.25202556724930614,
 0.2821878927682676,
 0.28459893803962466,
 0.3334443128443745,
 0.35180450928706347,
 0.39833046419903845,
 0.4030809727203302,
 0.43172031714050735,
 0.44505516608453566,
 0.4571954048781128,
 0.4724183121673574,
 0.4842518807476195,
 0.4898349348465986,
 0.4898832279800531,
 0.5214929939776869,
 0.5256041219089893,
 0.5281397049977173,
 0.5453943685213934,
 0.5495407647230078,
 0.56097874256186,
 0.5873188558853589,
 0.5882349550434885,
 0.5932134334956265,
 0.5941539012143098,
 0.600355449974797,
 0.622409309883855,
 0.6243489955015163,
 0.6362929813220939,
 0.6444983833145653,
 0.645010399

In [72]:
def test_feature(test_fea):
    param['n_jobs']=-1
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
   
    train_y = target
    oof_train = np.zeros((len(train),1))
    oof_test = np.zeros((len(test),1))
    
    train.reset_index(drop=True,inplace=True)
    for idx,(idx_trn,idx_val) in enumerate(kf.split(train,train['outliers'])):
        print('第 %d fold'%idx)
        tr_x,tr_y,val_x,val_y=train.iloc[idx_trn][test_fea],train['target'].iloc[idx_trn],train.iloc[idx_val][test_fea],train['target'][idx_val]

        trn_data = lgb.Dataset(tr_x,tr_y)#, categorical_feature=categorical_feats)
        val_data = lgb.Dataset(val_x,val_y)#, categorical_feature=categorical_feats)

        num_round = 10000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 500)
        oof_train[idx_val] = clf.predict(val_x, num_iteration=clf.best_iteration).reshape(-1,1)
        oof_test += clf.predict(test[test_fea], num_iteration=clf.best_iteration).reshape(-1,1)/5
        
        

    loss=np.sqrt(mean_squared_error(oof_train.reshape(-1), target))
    
    print('mean loss %f'%loss)
    if loss<3.650:
        train_prob=pd.DataFrame(oof_train)
        train_prob.columns=['class1']
        train_prob.to_csv("oof/train_prob_%s.csv"%loss,index=False)

        test_prob=pd.DataFrame(oof_test)
        test_prob.columns=['class1']
        test_prob.to_csv("oof/test_prob_%s.csv"%loss,index=False)


    return loss


test_dict={}
for idx,i in enumerate(thr_list):
    if idx>50:
        break
    print(i)
    bed_fea=set(scores_df[scores_df['gain_score']<=i]['feature'].tolist())
    bed_fea=[i for i in bed_fea]
    print(bed_fea)
    del_col =  ['card_id', 'first_active_month','target','outliers']+bed_fea
    df_test_fea = [c for c in features if c not in del_col ]
    test_dict[''.join(bed_fea)]=test_feature(df_test_fea)


-23.025850929940457
['hist_first_year', 'new_quarter_nunique', 'hist_now_year', 'hist_installments_min', 'new_category_3_p_mean_median']
第 0 fold
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55337	valid_1's rmse: 3.67871
[400]	training's rmse: 3.45379	valid_1's rmse: 3.65649
[600]	training's rmse: 3.39127	valid_1's rmse: 3.64731
[800]	training's rmse: 3.34267	valid_1's rmse: 3.6449
[1000]	training's rmse: 3.30064	valid_1's rmse: 3.64332
[1200]	training's rmse: 3.26354	valid_1's rmse: 3.64335
[1400]	training's rmse: 3.22871	valid_1's rmse: 3.64295
[1600]	training's rmse: 3.1953	valid_1's rmse: 3.6423
[1800]	training's rmse: 3.16364	valid_1's rmse: 3.64232
[2000]	training's rmse: 3.13387	valid_1's rmse: 3.64285
[2200]	training's rmse: 3.10478	valid_1's rmse: 3.6433
Early stopping, best iteration is:
[1740]	training's rmse: 3.17281	valid_1's rmse: 3.64225
第 1 fold
Training until validation scores don't improve for 500 rounds.
[200]	training's rms

Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55338	valid_1's rmse: 3.6791
[400]	training's rmse: 3.45416	valid_1's rmse: 3.65668
[600]	training's rmse: 3.39032	valid_1's rmse: 3.64887
[800]	training's rmse: 3.34325	valid_1's rmse: 3.64499
[1000]	training's rmse: 3.30012	valid_1's rmse: 3.6432
[1200]	training's rmse: 3.26223	valid_1's rmse: 3.64304
[1400]	training's rmse: 3.22931	valid_1's rmse: 3.6431
Early stopping, best iteration is:
[1093]	training's rmse: 3.28272	valid_1's rmse: 3.64271
第 1 fold
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55633	valid_1's rmse: 3.67952
[400]	training's rmse: 3.45555	valid_1's rmse: 3.66006
[600]	training's rmse: 3.39291	valid_1's rmse: 3.65354
[800]	training's rmse: 3.34483	valid_1's rmse: 3.65055
[1000]	training's rmse: 3.30239	valid_1's rmse: 3.64876
[1200]	training's rmse: 3.26642	valid_1's rmse: 3.64762
[1400]	training's rmse: 3.2309	valid_1's rmse: 3.64736
[1

[2000]	training's rmse: 3.13688	valid_1's rmse: 3.64527
Early stopping, best iteration is:
[1637]	training's rmse: 3.191	valid_1's rmse: 3.645
第 2 fold
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.5505	valid_1's rmse: 3.68425
[400]	training's rmse: 3.44926	valid_1's rmse: 3.66001
[600]	training's rmse: 3.38558	valid_1's rmse: 3.6526
[800]	training's rmse: 3.33596	valid_1's rmse: 3.64908
[1000]	training's rmse: 3.29433	valid_1's rmse: 3.64869
[1200]	training's rmse: 3.25665	valid_1's rmse: 3.64769
[1400]	training's rmse: 3.22041	valid_1's rmse: 3.6484
[1600]	training's rmse: 3.18706	valid_1's rmse: 3.64821
Early stopping, best iteration is:
[1217]	training's rmse: 3.25305	valid_1's rmse: 3.64759
第 3 fold
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55261	valid_1's rmse: 3.68362
[400]	training's rmse: 3.44997	valid_1's rmse: 3.66193
[600]	training's rmse: 3.38553	valid_1's rmse: 3.65495
[800]	training's

[400]	training's rmse: 3.4476	valid_1's rmse: 3.657
[600]	training's rmse: 3.38456	valid_1's rmse: 3.64975
[800]	training's rmse: 3.33665	valid_1's rmse: 3.64631
[1000]	training's rmse: 3.29383	valid_1's rmse: 3.64554
[1200]	training's rmse: 3.25564	valid_1's rmse: 3.64473
[1400]	training's rmse: 3.21918	valid_1's rmse: 3.64519
[1600]	training's rmse: 3.18713	valid_1's rmse: 3.64503
Early stopping, best iteration is:
[1202]	training's rmse: 3.25514	valid_1's rmse: 3.64466
第 3 fold
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55227	valid_1's rmse: 3.68407
[400]	training's rmse: 3.45149	valid_1's rmse: 3.66026
[600]	training's rmse: 3.38746	valid_1's rmse: 3.65228
[800]	training's rmse: 3.33825	valid_1's rmse: 3.65015
[1000]	training's rmse: 3.29656	valid_1's rmse: 3.64913
[1200]	training's rmse: 3.25914	valid_1's rmse: 3.6479
[1400]	training's rmse: 3.22417	valid_1's rmse: 3.64746
[1600]	training's rmse: 3.18995	valid_1's rmse: 3.64732
[1800]	t

[600]	training's rmse: 3.38595	valid_1's rmse: 3.65262
[800]	training's rmse: 3.33713	valid_1's rmse: 3.6499
[1000]	training's rmse: 3.29554	valid_1's rmse: 3.6493
[1200]	training's rmse: 3.25736	valid_1's rmse: 3.64806
[1400]	training's rmse: 3.22047	valid_1's rmse: 3.64903
[1600]	training's rmse: 3.18829	valid_1's rmse: 3.64929
Early stopping, best iteration is:
[1218]	training's rmse: 3.25358	valid_1's rmse: 3.64799
第 3 fold
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55252	valid_1's rmse: 3.68504
[400]	training's rmse: 3.44997	valid_1's rmse: 3.6618
[600]	training's rmse: 3.38701	valid_1's rmse: 3.65416
[800]	training's rmse: 3.33853	valid_1's rmse: 3.65145
[1000]	training's rmse: 3.2955	valid_1's rmse: 3.64964
[1200]	training's rmse: 3.25706	valid_1's rmse: 3.64799
[1400]	training's rmse: 3.22239	valid_1's rmse: 3.64771
[1600]	training's rmse: 3.18869	valid_1's rmse: 3.64726
[1800]	training's rmse: 3.15892	valid_1's rmse: 3.64731
[2000]	

...

Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55412	valid_1's rmse: 3.67929
[400]	training's rmse: 3.45626	valid_1's rmse: 3.65645
[600]	training's rmse: 3.39418	valid_1's rmse: 3.64832
[800]	training's rmse: 3.34736	valid_1's rmse: 3.64508
[1000]	training's rmse: 3.30551	valid_1's rmse: 3.64383
[1200]	training's rmse: 3.26879	valid_1's rmse: 3.64321
[1400]	training's rmse: 3.23587	valid_1's rmse: 3.64353
[1600]	training's rmse: 3.2039	valid_1's rmse: 3.64305
[1800]	training's rmse: 3.17244	valid_1's rmse: 3.64366
[2000]	training's rmse: 3.14385	valid_1's rmse: 3.64452
Early stopping, best iteration is:
[1601]	training's rmse: 3.20374	valid_1's rmse: 3.64303
第 1 fold
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55621	valid_1's rmse: 3.67717
[400]	training's rmse: 3.45724	valid_1's rmse: 3.65811
[600]	training's rmse: 3.39569	valid_1's rmse: 3.65061
[800]	training's rmse: 3.34772	valid_1's rmse: 3.64811

[400]	training's rmse: 3.45357	valid_1's rmse: 3.64909
[600]	training's rmse: 3.38979	valid_1's rmse: 3.64303
[800]	training's rmse: 3.33942	valid_1's rmse: 3.64066
[1000]	training's rmse: 3.29693	valid_1's rmse: 3.64017
[1200]	training's rmse: 3.25796	valid_1's rmse: 3.64048
[1400]	training's rmse: 3.22258	valid_1's rmse: 3.64049
Early stopping, best iteration is:
[949]	training's rmse: 3.30793	valid_1's rmse: 3.63975
mean loss 3.643813
0.6450103990037221
['hist_dayofweek_p_mean_median', 'hist_state_id_p_mean_min', 'new_dayofweek_p_mean_std', 'hist_installments_median', 'hist_hour_p_mean_median', 'new_dayofweek_nunique', 'new_sleep_vs_p2p', 'hist_city_id_p_mean_median', 'hist_category_2_nunique', 'new_dayofweek_p_mean_min', 'hist_month_p_mean_std', 'hist_month_lag_median', 'new_month_p_mean_median', 'new_weekofyear_p_mean_median', 'hist_weekofyear_p_mean_median', 'new_quarter_p_mean_std', 'new_dayofweek_p_mean_mean', 'hist_state_id_nunique', 'new_weekofyear_p_mean_mean', 'new_category

[200]	training's rmse: 3.55276	valid_1's rmse: 3.6825
[400]	training's rmse: 3.45264	valid_1's rmse: 3.65773
[600]	training's rmse: 3.38968	valid_1's rmse: 3.64976
[800]	training's rmse: 3.34182	valid_1's rmse: 3.647
[1000]	training's rmse: 3.30192	valid_1's rmse: 3.64584
[1200]	training's rmse: 3.26466	valid_1's rmse: 3.64456
[1400]	training's rmse: 3.22869	valid_1's rmse: 3.64442
[1600]	training's rmse: 3.19682	valid_1's rmse: 3.64429
[1800]	training's rmse: 3.16456	valid_1's rmse: 3.64425
[2000]	training's rmse: 3.13522	valid_1's rmse: 3.64467
[2200]	training's rmse: 3.10657	valid_1's rmse: 3.64496
Early stopping, best iteration is:
[1747]	training's rmse: 3.17263	valid_1's rmse: 3.64403
第 3 fold
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55311	valid_1's rmse: 3.68247
[400]	training's rmse: 3.45246	valid_1's rmse: 3.66015
[600]	training's rmse: 3.38988	valid_1's rmse: 3.65332
[800]	training's rmse: 3.34091	valid_1's rmse: 3.65019
[1000]	t

Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55392	valid_1's rmse: 3.67954
[400]	training's rmse: 3.45646	valid_1's rmse: 3.65614
[600]	training's rmse: 3.39578	valid_1's rmse: 3.64838
[800]	training's rmse: 3.34884	valid_1's rmse: 3.64476
[1000]	training's rmse: 3.30671	valid_1's rmse: 3.6428
[1200]	training's rmse: 3.27101	valid_1's rmse: 3.64219
[1400]	training's rmse: 3.23739	valid_1's rmse: 3.64277
[1600]	training's rmse: 3.20473	valid_1's rmse: 3.64232
Early stopping, best iteration is:
[1198]	training's rmse: 3.27148	valid_1's rmse: 3.64214
第 1 fold
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.55623	valid_1's rmse: 3.67767
[400]	training's rmse: 3.45717	valid_1's rmse: 3.65733
[600]	training's rmse: 3.39388	valid_1's rmse: 3.64965
[800]	training's rmse: 3.34417	valid_1's rmse: 3.64617
[1000]	training's rmse: 3.30326	valid_1's rmse: 3.64522
[1200]	training's rmse: 3.26648	valid_1's rmse: 3.64479

[1400]	training's rmse: 3.22322	valid_1's rmse: 3.63772
[1600]	training's rmse: 3.18974	valid_1's rmse: 3.63852
[1800]	training's rmse: 3.15812	valid_1's rmse: 3.63832
Early stopping, best iteration is:
[1376]	training's rmse: 3.22724	valid_1's rmse: 3.6376
mean loss 3.642941
